In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
csv_file = "Resources/social_media_appended_etl.csv"
social_media_df = pd.read_csv(csv_file)
social_media_df.head()

,id,dates,facebook,twitter,instagram,google,youtube,reddit,device
0,1,2010-01,48.04,5.96,0.0,0.0,4.83,5.77,all
1,2,2010-02,40.45,7.71,0.0,0.0,5.04,5.28,all
2,3,2010-03,30.93,8.74,0.0,0.0,4.41,6.31,all
3,4,2010-04,35.18,7.38,0.0,0.0,3.13,5.48,all
4,5,2010-05,39.62,6.84,0.0,0.0,3.46,5.89,all


In [3]:
#social_media_df = social_media_df.set_index(['id'])
#social_media_df

# after some exmaination, there was no need to set index on the id column, as later on for pushing the data frames into the sql,
# it assumed the first row of the index was null and that would violate primary key constariant. 

In [4]:
csv_file_retail = "Resources/retail_appended_etl.csv"
retail_data_df = pd.read_csv(csv_file_retail)
retail_data_df.head()

,id,dates,retail_sales_total,electronic_shopping_mail_order_houses,electronics_appliance_stores,clothing_stores,hobby_toy_game_stores
0,25,2012-01,315540,25932,7832,10686,1072
1,26,2012-02,331470,26420,8256,12618,1192
2,27,2012-03,368502,26365,8507,15205,1270
3,28,2012-04,349194,26528,7241,14003,1156
4,29,2012-05,373129,26921,7955,14593,1186


In [5]:
#retail_data_df = retail_data_df.set_index(['id'])
#retail_data_df

In [16]:
#merging two data frames in pandas
merged_df = pd.merge(retail_data_df, social_media_df, how ='inner')
merged_df.head()

,id,dates,retail_sales_total,electronic_shopping_mail_order_houses,electronics_appliance_stores,clothing_stores,hobby_toy_game_stores,facebook,twitter,instagram,google,youtube,reddit,device
0,25,2012-01,315540,25932,7832,10686,1072,50.06,5.58,0.0,0.0,6.16,5.95,all
1,26,2012-02,331470,26420,8256,12618,1192,50.55,5.87,0.0,0.0,6.42,6.65,all
2,27,2012-03,368502,26365,8507,15205,1270,51.03,5.78,0.0,0.0,6.64,5.93,all
3,28,2012-04,349194,26528,7241,14003,1156,52.44,5.51,0.0,0.0,5.84,4.32,all
4,29,2012-05,373129,26921,7955,14593,1186,50.90,5.18,0.0,0.0,9.03,5.81,all


In [7]:
#checking the data types for creating the table on pgadmin
retail_data_df.dtypes

id                                        int64
dates                                    object
retail_sales_total                        int64
electronic_shopping_mail_order_houses     int64
electronics_appliance_stores              int64
clothing_stores                           int64
hobby_toy_game_stores                     int64
dtype: object

In [9]:
#Connect to local database
rds_connection_string = "postgres:Venusfafar69@localhost:5432/socialretail_db"
engine = create_engine(f"postgresql://{rds_connection_string}")

In [10]:
#checking for tables
engine.table_names()

['social_media', 'retail']

In [11]:
social_media_df.to_sql(name='social_media', con=engine, if_exists='append', index=False)

In [12]:
retail_data_df.to_sql(name='retail', con=engine, if_exists='append', index=False)

In [15]:
#socila media table
pd.read_sql_query('select * from social_media', con=engine).head()

,id,dates,facebook,twitter,instagram,google,youtube,reddit,device
0,1,2010-01,48.04,5.96,0.0,0.0,4.83,5.77,all
1,2,2010-02,40.45,7.71,0.0,0.0,5.04,5.28,all
2,3,2010-03,30.93,8.74,0.0,0.0,4.41,6.31,all
3,4,2010-04,35.18,7.38,0.0,0.0,3.13,5.48,all
4,5,2010-05,39.62,6.84,0.0,0.0,3.46,5.89,all


In [14]:
#retaile table
pd.read_sql_query('select * from retail', con=engine).head()

,id,dates,retail_sales_total,electronic_shopping_mail_order_houses,electronics_appliance_stores,clothing_stores,hobby_toy_game_stores
0,25,2012-01,315540,25932,7832,10686,1072
1,26,2012-02,331470,26420,8256,12618,1192
2,27,2012-03,368502,26365,8507,15205,1270
3,28,2012-04,349194,26528,7241,14003,1156
4,29,2012-05,373129,26921,7955,14593,1186


In [33]:
#getting max socila media usage grouped by devices
pd.read_sql_query('SELECT device, MAX(facebook) AS max_facebook, MAX(twitter) AS max_twitter, MAX(instagram) AS max_instagram, MAX(google) AS max_google, MAX(youtube) AS max_youtube, MAX(reddit) AS max_reddit FROM social_media GROUP BY device ORDER BY max_facebook DESC;' , con=engine)

,device,max_facebook,max_twitter,max_instagram,max_google,max_youtube,max_reddit
0,console,96.00,71.36,19.86,2.73,32.05,90.36
1,desktop,91.65,27.21,2.48,1.04,14.99,39.30
2,mobile,86.21,37.61,15.79,0.36,5.47,21.50
3,tablet,81.82,9.32,2.11,0.13,2.66,2.56
4,all,79.28,25.69,13.73,0.35,13.86,11.90


In [44]:
#checking to see if this max usage was correspondent to gamestop stock shortage
pd.read_sql_query('SELECT dates FROM social_media WHERE reddit = 90.36;', con =engine)

,dates
0,2019-04


In [39]:
#joining the two tables
pd.read_sql_query('SELECT social_media.dates, social_media.facebook, social_media.twitter, social_media.instagram, retail.retail_sales_total, retail.clothing_stores FROM social_media INNER JOIN retail ON social_media.dates = retail.dates;', con = engine)

,dates,facebook,twitter,instagram,retail_sales_total,clothing_stores
0,2012-01,50.06,5.58,0.00,315540,10686
1,2012-02,50.55,5.87,0.00,331470,12618
2,2012-03,51.03,5.78,0.00,368502,15205
3,2012-04,52.44,5.51,0.00,349194,14003
4,2012-05,50.90,5.18,0.00,373129,14593
...,...,...,...,...,...,...
582,2021-07,80.43,6.71,3.95,558031,17876
583,2021-08,77.79,8.23,3.89,557018,17279
584,2021-09,72.81,9.58,5.39,536229,15846
585,2021-10,74.59,8.07,5.91,559399,17187
